In [3]:
import random
import operator
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools


pset = gp.PrimitiveSet("main", 9)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
a=4
pset.addTerminal(4)


pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


expr = gp.genFull(pset, min_=1, max_=5)
tree = gp.PrimitiveTree(expr)

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)



toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



x=toolbox.population()
print(x[1][1])
print(x[9][9])

tree = gp.PrimitiveTree(expr)




E:\Sofwares\Anaconda\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


max(max(add(max(bn, se), max(nn, se)), max(max(bc, bn), mul(se, thickness))), max(add(add(4, size), add(ma, shape)), mul(max(bn, mitoses), mul(size, shape))))
add(max(mul(mitoses, size), mul(se, se)), add(mul(4, thickness), max(4, se)))


In [18]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
#display(data)
data[6] =  pd.to_numeric(data[6], errors='coerce') 
# drop nas
data = data.dropna()

x_features = data.iloc[:,1:10]
y_features = data.iloc[:,10]
y_list = list(y_features.values.T.ravel())
x_array = x_features.values

scaler = preprocessing.StandardScaler()
scaled_X = scaler.fit_transform(x_features)


x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=42,shuffle = True)
np.mean(x_train)
np.mean(y_)


list42=[]
tree_no=0
fitnessM=[]
for t1 in range(10):
    for t2 in range(10):
        tree_no=tree_no+1
        print('tree : '+str(tree_no))
        function = gp.compile(str(x[t1][t2]), pset)
        for i in range(30):
            if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
                list42.append(4)
            else:
                list42.append(2)
        print("Batch 1 : ")
        print(list42)
        print("\n")

        scaled_x_list = []
        fitnessS=[]
        for j in range(12):
            tn=0
            tp=0
            fn=0
            fp=0
            x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
            listRand=[]
            scaled_X = scaler.fit_transform(x_train)
            scaled_x_list.append(scaled_X)
            for i in range(30):
                if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
                    listRand.append(2)
                else:
                    listRand.append(4)
            print("Batch "+str(j+2)+" : ")

            for n in range(30):
                if(listRand[n]==y_train[n]):
                    if(y_train[n]==2):
                        tn=tn+1
                    else:
                        tp=tp+1
                else:
                    if(listRand[n]==2):
                        fn=fn+1
                    else:
                        fp=fp+1



            print(listRand)
            print(y_train)
            print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
            prec=tp/(tp+fp)
            recall=tp/(tp+fn)
            fitness=(2*prec*recall)/(prec+recall)
            fitnessS.append(fitness)
            print('Fitness = '+str(fitness*100))
            print("\n")
        fitnessM.append(fitnessS*100)
        # print(x_train)
print(scaled_x_list)

print(fitnessM)


[[10.  5.  8. 10.  3. 10.  5.  1.  3.]
 [10.  5.  5.  3.  6.  7.  7. 10.  1.]
 [ 1.  1.  1.  1.  2.  1.  2.  1.  1.]
 [ 5.  1.  1.  1.  2.  1.  1.  1.  1.]
 [ 5.  6.  7.  8.  8. 10.  3. 10.  3.]
 [ 8. 10. 10. 10.  8. 10. 10.  7.  3.]
 [ 5.  1.  1.  4.  2.  1.  3.  1.  1.]
 [ 1.  2.  3.  1.  2.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  2.  1.  2.  1.  1.]
 [ 8. 10. 10. 10.  7.  5.  4.  8.  7.]
 [ 8. 10. 10.  7. 10. 10.  7.  3.  8.]
 [ 3.  1.  1.  1.  2.  1.  3.  1.  1.]
 [ 5.  1.  1.  1.  2.  1.  1.  1.  1.]
 [ 5. 10.  8. 10.  8. 10.  3.  6.  3.]
 [ 2.  1.  1.  1.  2.  1.  1.  1.  1.]
 [ 2.  3.  4.  4.  2.  5.  2.  5.  1.]
 [ 7.  6.  6.  3.  2. 10.  7.  1.  1.]
 [ 2.  1.  1.  2.  3.  1.  2.  1.  1.]
 [ 4.  1.  1.  1.  2.  1.  1.  1.  1.]
 [ 7.  6.  4.  8. 10. 10.  9.  5.  3.]
 [ 1.  1.  1.  2.  2.  1.  3.  1.  1.]
 [ 5.  3.  2.  4.  2.  1.  1.  1.  1.]
 [ 5.  3.  5.  1.  8. 10.  5.  3.  1.]
 [ 6.  2.  3.  1.  2.  1.  1.  1.  1.]
 [ 7.  3.  2. 10.  5. 10.  5.  4.  4.]
 [ 9.  4.  5. 10.  6. 10.

Batch 8 : 
[4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 4]
[4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 2]
FP : 1 
 FN : 1 
 TP : 11 
 TN : 17 
 
Fitness = 91.66666666666666


Batch 9 : 
[2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 4]
[2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 4]
FP : 1 
 FN : 0 
 TP : 11 
 TN : 18 
 
Fitness = 95.65217391304348


Batch 10 : 
[2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 4, 2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 4, 2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2]
FP : 0 
 FN : 0 
 TP : 9 
 TN : 21 
 
Fitness = 100.0


Batch 11 : 
[2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 2, 2, 4, 4, 2, 2]
[2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 4, 2, 2, 4, 4, 2, 2]
FP : 2 
 FN : 1 
 TP : 11 
 TN :

[2, 2, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4]
FP : 20 
 FN : 2 
 TP : 8 
 TN : 0 
 
Fitness = 42.105263157894726


Batch 5 : 
[4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2]
[2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4]
FP : 18 
 FN : 4 
 TP : 7 
 TN : 1 
 
Fitness = 38.88888888888889


Batch 6 : 
[4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 4]
[2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 4, 2, 2, 4, 4, 2, 2]
FP : 17 
 FN : 4 
 TP : 8 
 TN : 1 
 
Fitness = 43.24324324324324


Batch 7 : 
[2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 2, 4, 4, 4, 2, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 2]
[4, 4, 2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 4]
FP : 16 
 FN : 5 
 TP : 6 
 TN : 3 
 
Fitness = 36.36363636363636


Batch 8 : 
[4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 2, 

Batch 11 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2]
[2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 2]
FP : 21 
 FN : 0 
 TP : 8 
 TN : 1 
 
Fitness = 43.24324324324324


Batch 12 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4]
FP : 18 
 FN : 0 
 TP : 10 
 TN : 2 
 
Fitness = 52.63157894736842


Batch 13 : 
[4, 4, 4, 2, 4, 4, 2, 4, 2, 4, 2, 4, 4, 2, 4, 4, 4, 4, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2, 4, 2]
[2, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2]
FP : 7 
 FN : 0 
 TP : 12 
 TN : 11 
 
Fitness = 77.41935483870968


tree : 38
Batch 1 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

Batch 4 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 4, 4, 4, 2, 2]
FP : 18 
 FN : 0 
 TP : 11 
 TN : 1 
 
Fitness = 54.99999999999999


Batch 5 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2]
FP : 22 
 FN : 0 
 TP : 8 
 TN : 0 
 
Fitness = 42.10526315789474


Batch 6 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 2, 4, 2, 4, 2, 2, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 7 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2]
FP : 23 
 FN : 0 
 TP



Batch 3 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4]
[2, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 2, 4, 4, 4, 2, 4, 2, 4, 4, 2, 4]
FP : 14 
 FN : 5 
 TP : 11 
 TN : 0 
 
Fitness = 53.65853658536586


Batch 4 : 
[4, 2, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4, 4, 4, 4]
[4, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2]
FP : 17 
 FN : 5 
 TP : 8 
 TN : 0 
 
Fitness = 42.10526315789474


Batch 5 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4]
FP : 18 
 FN : 1 
 TP : 10 
 TN : 1 
 
Fitness = 51.28205128205129


Batch 6 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4]
[4, 4, 2, 4, 2, 4, 4, 2, 2, 4, 4, 4, 4, 2, 4, 2, 4, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4]
FP : 13 
 FN : 3 
 T

Batch 11 : 
[4, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4]
[4, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4]
FP : 1 
 FN : 0 
 TP : 14 
 TN : 15 
 
Fitness = 96.55172413793103


Batch 12 : 
[4, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 4]
[4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 2]
FP : 3 
 FN : 0 
 TP : 12 
 TN : 15 
 
Fitness = 88.8888888888889


Batch 13 : 
[2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2]
[2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2]
FP : 4 
 FN : 0 
 TP : 9 
 TN : 17 
 
Fitness = 81.81818181818181


tree : 71
Batch 1 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

Batch 2 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 4, 2, 4, 4]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 3 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4]
FP : 20 
 FN : 0 
 TP : 10 
 TN : 0 
 
Fitness = 50.0


Batch 4 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 4, 2, 4, 4, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 4, 2, 4, 4]
FP : 17 
 FN : 0 
 TP : 13 
 TN : 0 
 
Fitness = 60.46511627906976


Batch 5 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2]
FP : 22 
 FN : 0 
 TP : 8 
 TN : 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
scaled_X_test = scaler.fit_transform(x_test)
listRandtest=[]
function = gp.compile(str(x[0][1]), pset)
for i in range(len(y_test)-1):
    if(function(scaled_X_test[i,0],scaled_X_test[i,1],scaled_X_test[i,2],scaled_X_test[i,3],scaled_X_test[i,4],scaled_X_test[i,5],scaled_X_test[i,6],scaled_X_test[i,7],scaled_X_test[i,8])<0):
        listRandtest.append(2)
    else:
        listRandtest.append(4)
print(listRandtest)
print(y_test)

tn1=0
tp1=0
fn1=0
fp1=0


for q in range(50):
    if(listRandtest[q]==y_test[q]):
        if(y_test[q]==2):
            tn1=tn1+1
        else:
            tp1=tp1+1
    else:
        if(listRand[q]==2):
            fn1=fn1+1
        else:
            fp1=fp1+1


           
print("FP : "+str(fp1)+" \n "+ "FN : "+str(fn1)+" \n "+ "TP : "+str(tp1)+" \n "+"TN : "+str(tn1)+" \n ")
prec1=tp1/(tp1+fp1)
recall1=tp1/(tp1+fn1)
fitness1=(2*prec1*recall1)/(prec1+recall1)
print('Fitness = '+str(fitness1*100))


[2, 4, 2, 2, 4, 2, 2, 4, 4, 4, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 4, 4, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 4, 2, 4, 2, 4, 4, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 4, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 4, 2, 4, 4, 4, 4, 2, 2, 4, 2, 4, 4, 2, 2, 4, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 2, 4, 4, 4, 4, 4, 4, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 4, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 4, 

In [36]:
lsum_product = []; lsumx_product = [];  lsumy_product = [];
lpearson = []
for i in range(len(fitnessM)-1):
    x_mean = np.mean(fitnessM[i])
    y_mean = np.mean(fitnessM[i+1])
    for j,k in zip(fitnessM[i],fitnessM[i+1]):
        sub_x = j-x_mean; sub_y = k-y_mean
        product = (sub_x)*(sub_y)
        square_x = sub_x**2;square_y = sub_y**2
        
        lsum_product.append(product); lsumx_product.append(square_x); lsumy_product.append(square_y);
    print("np.sum(lsum_product) : ")
    x = np.sum(lsum_product)
    y = np.sqrt(np.sum(lsumx_product))
    z = np.sqrt(np.sum(lsumy_product))
    pearson_relation = x/(y*z)
    lpearson.append(pearson_relation**2)
print(lpearson)
    

np.sum(lsum_product) : 
0.09288224775692636
2.556296867368565
3.2142300878754173
np.sum(lsum_product) : 
0.21475560749193676
4.106814913277923
5.57524794481143
np.sum(lsum_product) : 
-4.803601002102428
6.133354980778662
7.491612730106689
np.sum(lsum_product) : 
-8.22180248150765
7.9157384350428694
7.656304772118844
np.sum(lsum_product) : 
-8.119541112110124
8.071781490952795
7.905505480990114
np.sum(lsum_product) : 
-12.421465068204796
8.308529989359313
8.643656497224477
np.sum(lsum_product) : 
-12.676061145233783
9.013736812006952
8.951099557893837
np.sum(lsum_product) : 
-10.921590872090789
9.308965408113062
10.233110557448153
np.sum(lsum_product) : 
-15.254541487464298
10.54756869402021
10.603763348260754
np.sum(lsum_product) : 
-15.074418939869405
10.90754100702979
11.422781004505993
np.sum(lsum_product) : 
-18.07254283225933
11.705322701703755
12.010786159093392
np.sum(lsum_product) : 
-24.587159632605907
12.27980609918527
12.785954067028566
np.sum(lsum_product) : 
-21.3352365342